# Propmt chaining

In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="qwen/qwen3-32b")

In [4]:
llm.invoke("What is the capital of France?")  # Example invocation

AIMessage(content="<think>\nOkay, the user is asking for the capital of France. Let me think. I remember that France is a country in Europe. Capitals are usually the major cities where the government is based. For France, I believe the capital is Paris. Wait, is there any other city that could be confused here? Maybe Lyon or Marseille? No, those are other major cities but not the capital. Let me confirm. I've heard Paris referred to as the capital in many contexts, like tourism, politics, etc. Also, the French government is headquartered there. Yeah, I'm pretty confident the answer is Paris. No need to overcomplicate it. The user probably just wants a straightforward answer. I'll go with Paris.\n</think>\n\nThe capital of France is **Paris**. It is the political, economic, and cultural center of the country and is renowned for its historical landmarks, art, and global influence. Let me know if you'd like additional details! 🇫🇷", additional_kwargs={}, response_metadata={'token_usage': {

In [5]:
result = llm.invoke("What is the capital of France?")  # Example invocation
result.content

'<think>\nOkay, so the user is asking, "What is the capital of France?" Let me think about how to approach this.\n\nFirst, I know that France is a country in Europe, and like many countries, it has a capital city. I remember from school that the capital of France is Paris. But wait, maybe I should double-check to make sure I\'m not confusing it with another country. For example, Germany\'s capital is Berlin, and Spain\'s is Madrid. Yeah, France is definitely Paris. \n\nWait, is there any chance that the capital could be a different city? Maybe Lyon or Marseille? No, Lyon is a major city in France, known for its historical significance and cultural aspects, but it\'s not the capital. Marseille is a large port city. The capital is definitely Paris. \n\nI should also consider if there\'s any recent change. Sometimes countries change capitals for political or administrative reasons, but I don\'t recall any such event in France recently. The government has been based in Paris for centuries.

In [7]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph,START,END
from IPython.display import display, Image

## Graph state 
class State(TypedDict):
    topic: str
    story: str
    improved_story: str
    final_story: str
    
    
def generate_story(state:State):
    msg = llm.invoke(f"generate a one sentence of a story about {state['topic']}")
    return {"story": msg.content}

def check_conflict(state:State):
    if "?"  in state["story"]:
        return "Fail"
    return "Pass"
def improved_story(state:State):
    msg= llm.invoke(f"Enhance the story with vevid details and make it more engaging: {state['story']}")
    return {"improved_story": msg.content}


def final_story(state:State):
    msg = llm.invoke(f"Add a Unexpected twist to this story:{state['improved_story']}")
    return {"final_story": msg.content}
    

In [10]:
## build the graph

graph = StateGraph(State)
graph.add_node("generate",generate_story)
# graph.add_node("check_conflict", check_conflict)
graph.add_node("improved_story", improved_story)
graph.add_node("final_story", final_story)


## Define  the edges  
graph.add_edge(START,"generate")
graph.add_conditional_edges("generate",check_conflict,{"Pass":"improved_story","Fail":"generate"})
graph.add_edge("improved_story","final_story")
graph.add_edge("final_story",END)


In [13]:
compile_graph = graph.compile()


graph_image = compile_graph.get_graph().draw_mermaid_png()
display(Image(graph_image))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`